# DistilBERT model
This notebook trains the DistilBERT model and exports a set of predictions for a test dataset.

**Trained on**: Waseem and Hovy (2016)
**Tested on**: SemEval (2019)

First we need to install the required packages.

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.4 MB/s 
     |████████████████████████████████| 6.6 MB 52.9 MB/s 
     |████████████████████████████████| 120 kB 81.2 MB/s 


In [2]:
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install wordsegment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 8.0 MB/s 


In [4]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 10.1 MB/s 
     |████████████████████████████████| 115 kB 89.8 MB/s 
     |████████████████████████████████| 212 kB 61.8 MB/s 
     |████████████████████████████████| 127 kB 57.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [6]:
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizer
import numpy as np
from datasets import load_metric, load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pickle
import torch
from google.colab import drive
import preprocessor as p
import html
from torch.utils.data import DataLoader
import torch.nn.functional as nn


# Mount drive for loading the datasets
drive.mount('/content/drive')
import sys
sys.path.insert(0, '/content/drive/MyDrive/Colab Notebooks/')

from reader import Reader

FILENAME = "drive/MyDrive/Colab Notebooks/data/sem_eval_all.pkl"
TEMPERATURE = 1.468

Mounted at /content/drive


## Split and tokenize the datasets

In [8]:
class HateDataset(torch.utils.data.Dataset):
    """Dataset class used for combining the data encodings and labels."""
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
X, y = Reader.load(FILENAME)
X = Reader.preprocess(X)

mapping = {'hate': 1, 'none': 0}
y = [mapping[b] for b in y]

X_train, X_val, X_test, y_train, y_val, y_test = Reader.split_with_validation(X, y)

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize all datasets
train_encodings = tokenizer(X_train, truncation=True, padding=True)
val_encodings = tokenizer(X_val, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

# Combine the encodings with the labels to Torch datasets
train_dataset = HateDataset(train_encodings, y_train)
val_dataset = HateDataset(val_encodings, y_val)
test_dataset = HateDataset(test_encodings, y_test)

## Load existing model

In [10]:
model = torch.load(F"drive/MyDrive/Colab Notebooks/output/distilbert-waseem-hovy-calibrated.pth")
model.eval()

ModelWithTemperature(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (

## Run model on the test dataset
The predicted output logits of the test dataset are converted to class probabilities using Softmax.

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

logits = []
labels = []

test_loader = DataLoader(test_dataset)

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits.append(outputs.tolist()[0])
        labels.append(label)

probabilities = nn.softmax(torch.div(torch.tensor(logits), TEMPERATURE), dim=1)
print(probabilities.tolist()[:10])

[[0.9557445049285889, 0.04425544664263725], [0.9194495677947998, 0.08055050671100616], [0.11756302416324615, 0.8824369311332703], [0.858680009841919, 0.14132001996040344], [0.19634830951690674, 0.8036516904830933], [0.8502442836761475, 0.14975565671920776], [0.8443337678909302, 0.15566617250442505], [0.8504765033721924, 0.1495235115289688], [0.13572950661182404, 0.8642705082893372], [0.8906922340393066, 0.10930778831243515]]


In [12]:
predictions = torch.argmax(probabilities, dim=-1)
metric = load_metric("accuracy")
accuracy = metric.compute(predictions=predictions, references=labels)
print("Test dataset: ", accuracy)

Test dataset:  {'accuracy': 0.6404166666666666}


## Export predictions

In [13]:
import numpy as np

def to_predictions(predictions, labels):
    predicted_classes = np.argmax(predictions, 1)
    return list(map(lambda predicted_values, predicted_class, actual_class:
                    {'predicted_class': class_name(predicted_class),
                     'actual_class': class_name(actual_class),
                     'predicted_value': predicted_values[predicted_class].item(),
                     'text': None},
                    predictions, predicted_classes, labels))
def class_name(index):
    if index == 0:
        return "None"
    elif index == 1:
        return "Hate"

predictions_info = to_predictions(probabilities, labels)
pickle.dump(predictions_info, open("drive/MyDrive/Colab Notebooks/output/distilbert-waseem-semeval.p", "wb"))